# <center>LORA微调原理&实战

前言
<br>



&emsp;&emsp;微调（Fine-Tuning） 是机器学习特别是深度学习领域中的一种技术。它的核心概念是在一个已经训练好的大模型的基础上，通过在特定任务上进行额外的训练来优化模型性能。微调通常用于在已有的大型预训练模型上进行特定领域的任务优化，而不必从头开始训练整个模型，并在具体任务上获得更好的表现。

目录

一 LORA原理浅析<br>
&emsp;1 为什么要微调<br>
&emsp;2 LORA原理介绍<br>
二 基于ChatGLM3-6b进行LORA微调实战<br>
&emsp;1 微调环境准备<br>
&emsp;2 微调数据下载与预处理<br>
&emsp;3 执行LORA微调<br>
&emsp;4 使用、合并LORA微调后的模型<br>
&emsp;5 微调效果评测<br>

# 1. LORA原理浅析

## 1.1 为什么要微调

&emsp;&emsp;**缩放法则**（Scaling Law）增加语言模型的参数数量、数据量和训练时间，能够显著提升模型在自然语言处理任务上的性能。通常情况下，模型的错误率、困惑度（perplexity）等评价指标会随着模型规模的指数级增长呈现出对数下降的趋势。也就是说，模型的性能与参数数量之间的关系是对数线性的。模型的涌现能力也随着LLM参数的提升而出现。

&emsp;&emsp;虽然扩展模型规模能提高性能，但随之而来的计算成本和时间成本也会急剧增加，特别是在硬件受限的情况下，可能无法简单地通过扩大规模来提高性能。自然语言处理中的许多应用程序都依赖于使一个大规模的、预先训练过的语言模型适应于多个下游应用程序。这种自适应通常是通过微调来完成的，即更新预训练模型的所有参数。微调的主要缺点是，新模型包含的参数与原始模型一样多（GPT-3有1750亿个可训练参数）。小公司或个人只能对大模型进行微调后使用。

&emsp;&emsp;对于大模型微调方法，我们本质追求的就是通过对少量参数的高效修改，来最大化地影响模型的原始参数，以此达到针对特定下游任务的最佳适配。Lora（Low-Rank Adaptation）作为当前流行且关键的方法论，提供了一种不同的途径来实现这一目标。Lora的核心思想和方法论其实具有广泛的通用性，它不仅适用于大语言模型的微调，还在像'Stable Diffusion'这样的图形应用中大量应用了Lora技术来生成特定风格的AI绘图。因此本文我们就来详细的剖析一下Lora的原理以及它是如何应用于大模型微调领域的。

## 1.2 LORA原理介绍

&emsp;&emsp;**过参数化模型**（Over-parametrized models）是指参数数量远超训练数据样本数量的模型。换句话说，模型中可调参数的数量大于用于训练的样本数量。如GPT系列模型和transformer模型均为此类模型。采用这种策略可以带来性能上的大幅提升。

&emsp;&emsp;同样的，如此大的参数量，一定是有冗余的，也就是说：如此多的参数对于很多下游任务，不一定都是有用的。垂直领域的知识所需要的参数一定远小于全量参数，对这样的任务，可能仅仅一部分重要的参数就可以做的足够好。实际上存在于一个较低的内在维度上。假设模型适应过程中权重的变化也具有较低的“内在秩”，基于此这导致了低秩适应（LoRA）方法的灵感。LoRA允许我们通过优化适应过程中密集层变化的秩分解矩阵来间接训练神经网络中的一些密集层，同时保持预先训练的权值冻结，以实现微调效果并节省了计算资源。

&emsp;&emsp;通俗来讲LoRA的策略是：通过使用较小规模的矩阵来近似模拟大模型的原始矩阵。实现的方式是基于低秩分解的数学原理，通过较少的参数更新实现对大模型复杂功能的有效捕捉和适配，在减少计算资源消耗和提升微调效率的同时，保持或甚至提升模型对特定任务的适应性和性能。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240905114033858.png" width=80%></div>

&emsp;&emsp;如图所示，左边是采用全参数更新的微调方法，右边是采用Lora的微调方法。以ChatGLM3-6b模型为例，它有60亿个参数，这些参数被表示在一个权重矩阵W中。在微调过程中为了最小化损失函数，我们会计算一个ΔW矩阵，里面包含了对原始权重的更新信息。

&emsp;&emsp;LORA通ΔW的分解减少了所需计算的参数，即用两个较小的LoRA矩阵A和B来表示大矩阵ΔW。如果A的行数与ΔW相同，B的列数与ΔW相同，我们可以将分解写成ΔW = AB。（AB是矩阵A和B的矩阵乘积结果。）

&emsp;&emsp;我们可以做一下简单计算来直观感受Lora方法节省的内存：设ΔW有20，000行、30，000列，则它有6亿个参数。我们选择超参数r为8，那么实际所需的参数量为20，000x8 + 8x30，000 = 400，000
，这是全参数量ΔW的1/1500.由此可见LORA的优化效果是非常可观的。这也让单卡进行微调训练成为可能。


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240905110121940.png" width=40%></div>

以上这张图来源微软团队的文章：《LORA: LOW-RANK ADAPTATION OF LARGE LANGUAGE MODELS》 ，地址：https://arxiv.org/pdf/2106.09685.pdf 

&emsp;&emsp;LoRA在涉及矩阵乘法的模块中引入两个低秩矩阵A和B（更新矩阵）来模拟全量微调的过程。

1. $x$：局部的某一个输入，维度为$d$；
2. 左侧蓝色框：大模型某一部分的原始权重，其参数$W$是一个 $d \times d$的高维空间；
3. 右侧橘色框：两个低维的矩阵空间$A$和$B$，也叫做更新矩阵，是LoRA引入的两个新矩阵；

&emsp;&emsp;LoRA并不是直接去微调预训练模型权重中的参数，而是仅优化低秩分解的矩阵，产生近似于全量微调的效果，因为当$r = d$时，实质上就是全量微调的效果。在实践中，$r$的值非常小就足够了，这样能够在对性能影响最小的情况下显著降低计算和内存成本。而将$B$初始化为零，是为了可以确保用模型的原始预训练权重开始微调过程，在不断的训练过程中，但$A$、$B$更新矩阵产生了参数更新，就按照一个因子$α$进行缩放，然后将其添加到大模型的原始权重中。该因子的默认值为1，这意味着在计算大模型的前向传播时，预训练的权重和低秩权重更新的权重相等。当然也可以更改$α$的值以平衡原始权重和更新权重之间的关系。

&emsp;&emsp;其具体做法，如果我们进一步拆解，就是如下的这样一个过程：

- **架构设计**

&emsp;&emsp;在整体的架构设计上，LoRA的核心是在原始的预训练大语言模型旁外挂一个附加的网络通路，这个网络通路的目的，就是通过两个矩阵A和B相乘得到的维度来模拟本征维度，即：用小矩阵模拟大矩阵。

> 本征维度（Intrinsic Dimension）：是指数据或空间中所需的最小维度，以便充分描述其中的结构或特征。换句话说，尽管数据可能存在于高维空间中，但其实际所包含的信息可能集中在一个更低维度的子空间内，本征维度就是描述这个低维子空间的维度。


本征维度实例说明:

图像数据：一张高分辨率的彩色图像可能有数百万个像素点（高维度），但实际内容（如物体的形状、颜色等）可能只需要几十到几百个参数来描述。因此，图像的数据本征维度远小于其表面维度。

金融数据：在分析股票市场时，可能会考虑上百种指标，但实际影响市场走势的关键因素可能只有少数几个。

几何空间：假设我们有一些数据点在三维空间中分布，但这些数据点实际上全部位于一个二维平面上。尽管数据处于三维空间中，但由于它们仅在二维平面上变化，因此它们的本征维度是 2，而不是 3。



&emsp;&emsp;当训练一个神经网络时，一般来说是经历这样的阶段：对于给定的数据集，设计好神经网络结构，对神经网络中的参数进行随机的初始化，执行训练过程使得Loss越来越低。对于一个参数量为D的神经网络，训练的根本目的就是在D维空间中，根据训练数据找到拟合的最优解，在上述论文中就提出，并不是所有的D都是有效的，是存在冗余的，所以只要找到其中的d个参数，就可以拟合出该神经网络的最优解，所以这个d，就是所谓的该神经网络的本征维度。

&emsp;&emsp;看一下如下公式：

$$h = W_0x + \Delta Wx = W_0x + BAx$$

&emsp;&emsp;$W_0$表示的就是`Pretrained Weights`，$\Delta W$就是通过特定的下游任务的训练集所对原来的模型权重矩阵（$d \times d$）高维空间，去调整其中一部分的模型权重。而LoRA，把需要进行微调的参数 $\Delta W $ 从一个$d \times d$的高维空间， 转化成$B \times A \times x$这样一个矩阵的相乘过程，其总体的参数值与 $\Delta W$ 等价。

&emsp;&emsp;至此，如何让$x$ 和$B$、$A$两个小矩阵相乘得到 $\Delta Wx$，这成为了LoRA要解决的核心问题。

- **输入输出维度设计**

&emsp;&emsp;用两个小的矩阵去模拟大矩阵，简化的是中间的计算过程，接受的输入和输出维度，还是要和原始模型权重的维度相同，所以要设计$A$矩阵的输入和$B$矩阵的输出的维度均为d，才能与预训练模型层的原有的维度保持一致。

$$W_0 \in R^{d \times k}, B \in R^{d \times r}, A \in R^{r \times k}$$

&emsp;&emsp;通过上述的维度设计可以看到，原来的预训练模型的权重矩阵是$d \times k$ 的大小，那$A$和$B$相乘后，最终输出的维度同样还是 $d \times k$。当维度保持一致后，便可以保证矩阵运算的正常执行，接下来就进入到最关键的概念：低秩分解。

- **低秩分解**

&emsp;&emsp;矩阵$A$先将输入的$d$维数据降维至$r$维，$r$，就是一个矩阵的秩。它的数学含义就是：现在有一个高维的空间，如果这个高维空间是有冗余的，那么一定可以通过降维的方式降低它的维度，降到不能再降为止，得到最终的这个维度，就称为该矩阵的秩。因为$r$ << $d$，所以在微调/训练的过程中，矩阵计算的量就会大幅度的降低，从 $d \times d$ 变为 $ d \times r + r \times d$ 。
同时因为是小矩阵（$A$），所以参数量也会大幅度降低。如果将$r$设置为$d$的十分之一，百分之一甚至是千分之一，实质上就是在显著降低参数的数量级，这也就是为什么使用LoRA对大语言模型进行微调时，整个涉及的参数量只有原始参数量的千分之几。就是通过低秩分解这个过程来达到大幅降低模型的参数量和计算量的目的。

- **维度映射**

&emsp;&emsp;最后，$B$矩阵将$r$维的参数矩阵再映射回$d$维，从而与预训练模型的原始矩阵维度保持一致。

&emsp;&emsp;整个训练过程是：原始权重矩阵保持冻结状态，并且不再接受任何进一步的调整，仅训练$A$和$B$，最终使用微调后的模型时，将原始权重和适应后的权重进行合并，从而达到对特定数据集的参数调整的目的。

LoRA 的核心思想是通过低秩矩阵分解 
A×B 来逼近全权重更新 
ΔW。如果选择的秩 
𝑟
接近或等于该任务的本征维度，那么低秩分解能够更好地捕捉到该任务的关键模式和特征，从而提高微调效果。

当 
𝑟
 接近本征维度时：模型的权重更新空间能有效捕捉任务相关的变化，得到更优的微调结果。这时，低秩矩阵分解能够最大限度地逼近真正的权重更新矩阵 
ΔW，而且计算效率相比全权重微调要高很多。

当 
𝑟
 小于本征维度时：低秩矩阵的表达能力不足，可能无法很好地捕捉任务中的重要特征或模式。此时，模型的微调效果可能不如预期。

当 
𝑟
 大于本征维度时：此时矩阵分解已经能够很好地逼近原始权重更新矩阵，但增加的计算成本可能没有实际收益，甚至可能引入过拟合的风险。

>理想情况下，r 应该接近任务的本征维度，以在不损失模型性能的前提下，实现高效的微调。然而，确定本征维度往往是困难的，因为它依赖于任务、数据集和模型架构。在实际操作中，通常采用实验验证的方式来选择合适的 r。

&emsp;&emsp;由此，我们可以看到LoRA方法有如下的优势：
1. 可以对一个预先训练过的模型进行共享，并用于为不同的任务构建许多小型LoRA模块。我们可以通过替换矩阵A和矩阵B来冻结共享模型并有效地切换任务，从而显著地减少了存储需求和任务切换开销。
2. LoRA使训练更有效率，并将硬件进入的门槛大幅降低（将近3倍），因为不需要计算梯度或保持大多数参数的优化器状态。
3. LoRA简单的线性设计允许我们在部署时将可训练矩阵与冻结权值合并，通过构造，与完全微调的模型相比，不引入推理延迟。
4. LoRA与许多其他的微调的方法正交，可以与其中许多方法结合，比如前缀调优（p-tuning）。
5. 训练速度相较于全量微调更快。

- **扩展阅读**：
  
Practical Tips for Finetuning LLMs Using LoRA (Low-Rank Adaptation)：
https://magazine.sebastianraschka.com/p/practical-tips-for-finetuning-llms

MEASURING THE INTRINSIC DIMENSION OF OBJECTIVE LANDSCAPES：https://arxiv.org/pdf/1804.08838.pdf 

# 2. 基于ChatGLM3-6b进行LORA微调实战

## 2.1 微调环境准备

&emsp;&emsp;本次实验环境配置：
- 操作系统：Ubuntu 22.04；
- GPU：3090双卡，总共48G显存；
- CPU：AMD 5900X；
- 存储：64G内存+2T SSD数据盘；


官方硬件需求：显存：24GB及以上（推荐使用30系或A10等sm80架构以上的NVIDIA显卡进行尝试） 内存：16GB RAM: 2.9 /16 GB GPU RAM: 15.5/16.0 GB

本教程使用使用单张 GPU对 ChatGLM3-6B 数据集进行LoRA方法微调，**使其具备专业的广告生成能力**。项目地址：https://github.com/THUDM/ChatGLM3/blob/main/finetune_demo/lora_finetune.ipynb

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240905150634867.png" width=80%></div>

在微调环境准备阶段，需要先检查代码的运行地址，确保运行地址处于 finetune_demo 中。 并确保已经安装了 requirements.txt中全部依赖。
使用以下命令安装依赖：

pip install -r requirements.txt

这条命令会读取 `requirements.txt` 文件，并安装其中列出的所有依赖包。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240903104308880.png" width=80%></div>

- 其中mpi4py这个依赖，如果在正常状态`pip install`方法不能直接拉取，需要`conda install`的方式下载，会同时拉去前置的openmpi依赖。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240903115334033.png" width=50%></div>

## 2.2 微调数据下载与预处理

&emsp;&emsp;官网为**广告生成能力项目**提供了一个微调示例：AdvertiseGen 数据集，里面包含两个文件分别是训练集和测试集。可以进入Tsinghua Cloud：https://cloud.tsinghua.edu.cn/f/b3f119a008264b1cabd1/?dl=1 下载并上传到`finetune_demo/data`路径下。

- 模型路径大家要使用绝对路径。因为LoRA仅存储adapter部分参数，而adapter配置json文件记录了预训练模型的路径，用于读取预训练模型权重。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240903104941760.png" width=80%></div>

一种更便捷的方式就是在服务器终端使用`wget`命令来进行下载。同时下载到的AdvertiseGen数据集是一个.tar.gz的压缩文件，需要解压才可使用：
```bash
wget - O AdvertiseGen https://cloud.tsinghua.edu.cn/f/b3f119a008264b1cabd1/?dl=1
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401101712121.png" width=80%></div>

&emsp;&emsp;该数据集任务为根据输入（content）生成一段广告词（summary），其数据格式如下：


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240902143658551.png" width=80%></div>

```json
{"content": "类型#上衣*风格#街头*图案#创意*衣样式#卫衣", "summary": "在这件卫衣上，BRAND-white集合了女性化的柔美还有不变的街头风采，<UNK><UNK>的向日葵花朵美丽的出现在胸前和背后，犹如暗<UNK>闪光的星星一般耀眼又充满着<UNK>的生命力，而后品牌标志性的logo<UNK>出现，呈现出将花束固定的效果，有趣极了，穿的不仅是服饰。更是新颖创意的载体。"}
```

&emsp;&emsp;这里的数据集格式是无法直接进行模型识别的，因此我们需要修改成单轮对话的数据微调格式。可以直接在数据集所在的文件夹新建一个python脚本文件用来转化，代码内容如下。可以将这个转化脚本命名为trans.py,在需要进行微调的数据同一级的文件夹下编辑并保存该脚本，执行 python trans.py ，实现微调数据的转化，新的数据文件名称为`train2.json`.

In [ ]:
import json
import time
 
def process_and_save_json(input_filepath, output_filepath):
    # 记录开始时间
    start_time = time.time()   
    with open(input_filepath, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line.strip())
 
            user_data = {
                "role": "user",
                "content": data["content"]
            }
            assistant_data = {
                "role": "assistant",
                "content": data["summary"]
            }
            outfile = open(output_filepath, 'a', encoding='utf-8')
            json.dump({"conversations": [user_data, assistant_data]}, outfile, ensure_ascii=False)
            outfile.write('\n')
    # 记录结束时间
    end_time = time.time()
    elapsed_time = end_time - start_time
 
    print("耗时:", elapsed_time, "秒")
 
input_filepath = 'train.json'  # 传入处理前的json
output_filepath = 'train2.json'  # 处理后的保存位置
 
process_and_save_json(input_filepath, output_filepath)

&emsp;&emsp;执行后，数据格式如下：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/37eb19d70817262358e43ab7e3f98ba.png" width=80%></div>

- 转化结束后新文件名称是`xx2.json`文件，注意文件名称的修改或调用地址的确定。

## 2.3 执行LORA微调

接下来，我们只需在终端将配置好的参数传递给程序，便能够利用命令行工具高效地进行微调。
```bash
CUDA_VISIBLE_DEVICES=0 NCCL_P2P_DISABLE="1" NCCL_IB_DISABLE="1" python finetune_hf.py  data/AdvertiseGen  Models/LLM/chatglm3-6b  configs/lora.yaml

```

- Models/LLM/chatglm3-6b这里需要修改成具体的模型权重所在地址。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240905152326053.png" width=80%></div>

微调训练过程会持续一段时间，期间每500步便会记录生成一份权重，保存在`output`文件夹中.

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240903155733982.png" width=80%></div>

我们可以打开一个新的终端窗口通过`watch -n 1 nvidia-smi
`方法实时监控GPU显存使用情况。可以看到我们选择单卡训练，会占用20G的显存资源。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240903160457651.png" width=80%></div>

训练结束的界面如下：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240903164330681.png" width=80%></div>

## 2.4 使用、合并LORA微调后的模型

&emsp;&emsp;在微调过程顺利完成之后，我们可以在output目录中观察到多个以checkpoint-*命名的文件夹，这些文件夹对应于训练过程中的不同阶段，每个文件夹的名称通常包含了一个序号，表示模型训练到达的轮次。在这些文件夹中，最后一个创建的通常代表了训练的最终轮次，也就是我们微调的最终结果。
为了将模型部署到实际应用中，我们需要选择这个最后一轮次的微调权重。这可以通过加载对应的checkpoint-*文件夹中的模型权重文件来实现。一旦模型权重被加载，我们就可以使用这些权重来进行推理（inference），即将模型应用于新的数据上，以生成预测或执行任务

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240903164602259.png" width=80%></div>

通过以下代码可以选择使用最后一轮微调结果的输出的权重作为微调模型进行功能测试：
```bash
CUDA_VISIBLE_DEVICES=0 NCCL_P2P_DISABLE="1" NCCL_IB_DISABLE="1" python inference_hf.py output/checkpoint-3000/ --prompt "类型#裙*版型#显瘦*材质#网纱*风格#性感*裙型#百褶*裙下摆#压褶*裙长#连衣裙*裙衣门襟#拉链*裙衣门襟#套头*裙款式#拼接*裙款式#拉链*裙款式#木耳边*裙款式#抽褶*裙款式#不规则"


```


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240903164858312.png" width=80%></div>

## 2.5 微调效果评测

通过选择不同check point生成的权重文件进行功能测试，可以看到随着训练轮次的上升模型的效果也在逐步提升。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240903171621992.png" width=80%></div>

通过同样的方式进行其他问题的单轮对话也是可以正常交互的，并没有出现`灾难性遗忘`状态的出现。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240905163224175.png" width=80%></div>

我们可以进入lora的配置文件中进行查看,以下是部分主要参数的说明：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240903175815374.png" width=100%></div>

**data_config**部分
- **`train_file: train.json`**
  - 训练数据集文件，指定为`train.json`。
- **`val_file: dev.json`**
  - 验证数据集文件，指定为`dev.json`。通常用于评估模型在训练过程中的表现。
- **`test_file: dev.json`**
  - 测试数据集文件，指定为`dev.json`。在某些情况下，测试和验证数据集可以是相同的文件。
- **`num_proc: 16`**
  - 用于数据预处理的并行进程数，设定为 16。可以加快数据加载和处理的速度。

**模型输入输出长度**

- **`max_input_length: 256`**
  - 模型输入序列的最大长度。输入序列将被截断或填充至 256 个tokens。
- **`max_output_length: 512`**
  - 模型输出序列的最大长度。输出序列的长度被限制为 512 个tokens。

**traininng_args部分**

- **`output_dir: ./output`**
  - 训练输出结果的保存目录，指定为`./output`。
- **`max_steps: 3000`**
  - 最大训练步数，设定为 3000 步。
- **`learning_rate: 5e-5`**
  - 学习率，控制模型参数更新的速度，设定为 5e-5。
- **`per_device_train_batch_size: 4`**
  - 每个设备（GPU/TPU/CPU）的训练批量大小，设定为 4。
- **`dataloader_num_workers: 16`**
  - 数据加载器使用的工作进程数量，设定为 16。提高数据加载效率。
- **`remove_unused_columns: false`**
  - 是否在数据集中删除未使用的列。设为 `false`，保留所有列。

**检查点保存策略**

- **`save_strategy: steps`**
  - 保存检查点的策略，设为 `steps`，即按照步骤间隔保存。
- **`save_steps: 500`**
  - 每隔 500 步保存一次模型检查点

**LORA参数设置**

**`peft_type: LORA`**
这是指微调方法的类型。`LORA` 表示使用 Low-Rank Adaptation 方法。

**`task_type: CAUSAL_LM`**
这是指任务的类型。`CAUSAL_LM` 表示微调的任务是因果语言模型（Causal Language Model），也就是像 GPT 类的自回归模型，预测下一个词时只考虑前面的上下文。

**`r: 8`**
`r` 是低秩分解中的秩（rank），在 LoRA 中，它决定了用于近似原始权重矩阵的低秩矩阵的维度。`r=8` 表示使用秩为 8 的矩阵来分解和近似模型中的原始权重矩阵。

**`lora_alpha: 32`**
`lora_alpha` 是一个缩放因子，用于在训练过程中对更新的权重进行缩放。具体来说，`lora_alpha=32` 会将 LoRA 模块中计算得到的权重更新乘以 32，从而控制更新的幅度，避免梯度过大或过小。

**`lora_dropout: 0.1`**
`lora_dropout` 是指在 LoRA 模块中的 dropout 概率。`0.1` 表示在微调过程中有 10% 的神经元会被随机丢弃（dropout），这有助于防止过拟合，提升模型的泛化能力

α 是一个缩放因子，用来控制矩阵分解后更新权重的比例。具体来说，在实际更新权重矩阵时，LoRA 会通过 α 来缩放更新量，以控制权重更新的幅度：
过度调整。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/LingYu/image-20240905181100034.png" width=50%></div>

r：控制矩阵分解的秩，也就是两个矩阵 
𝐴
 和 
𝐵
 的中间维度。较大的 r 会导致 
𝐴
×
𝐵
 产生更多信息，表达能力更强，但同时会增加计算开销和内存占用。

α：用于缩放矩阵分解后产生的更新量。为了避免 
𝐴
×
𝐵
 的结果规模过大或过小，LoRA 引入了 α 进行缩放，确保更新的权重规模适合微调过程。

在分解的过程中，r 控制了模型的表达能力，而 α 则是一个平衡因子，确保更新的规模合理。更大的 r 提供了更多的更新自由度，但这会导致 
𝐴
×
𝐵
 的值变大，因此需要用较小的 α 来进行适当的缩放；反之，较小的 r 可能需要较大的 α 来补偿权重更新的不足。

这里与LoRA有关比较重要的参数是r和lora_alpha，其中选择LoRA的参数r是个需要实验的超参数，太大可能导致过拟合，太小可能不足以处理数据集中的任务多样性。一般来说alpha = r x 2 )是较好的选择，但在某些情况下，不同的 r 和 alpha 组合可能会产生更好的性能。

- 实验表明，如果为更多层启用LoRA，虽然内存需会略微增加，但模型性能有显著提升。

总结：

通过上述的测试不难看出，经过LoRA微调后的ChatGLM3模型还能够比较准确的保持其原有的通用知识生成能力，除此之外，LoRA的优势更在于其推理阶段的优势。因为它在推理阶段是直接使用训练好的A、B低秩矩阵去替换原预训练模型的对应参数，就可以避免因增加网络的深度所带来的推理延时和额外的计算量。所以特别适用于对推理速度和模型性能都有较高要求的应用场景。